<H1>PyCity School Analysis</H1>

This analysis combines two data sets - one with data on schools and their budgets, and another with data on students from those schools and their performance on reading and math tests.

The combined data set is then analyzed by school performance, with detailed looks at performanspending per student, 

* The highest perfoming schools by overall passing rate tended to be smaller, with roughly 1-2K students, while the bottom performing schools were all larger, with 2.9 - 4.7K students. The bottom performing schools are sometimes double the size of the higher performing schools. However, Medium sized schools have the higest overall passing rate.

* School math scores appear to be consistent from grade to grade, but vary a lot from school to school. 

* Schools that spent the least on a per student basis saw the best results in math, reading, and overall passing rates. 

In [425]:
#import dependencies
import pandas as pd

#file path variables
schools = "../resources/schools_complete.csv"
students = "../resources/students_complete.csv"

#read csvs
schools_df = pd.read_csv(schools)
students_df = pd.read_csv(students)

#merging the school and student data
merge_df = pd.merge(schools_df,students_df,how="left",on=["school_name","school_name"])
merge_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


<H1>District Summary</H1>

In [426]:
#group the data by school name
grouped_data = merge_df.groupby("school_name")

In [427]:
#Total number of unique schools
total_schools = len(grouped_data["school_name"].unique())

In [428]:
#Total students
total_students = grouped_data["size"].mean().sum()

In [429]:
#Total budget
total_budget = grouped_data["budget"].mean().sum()

In [430]:
#Average math score
math_avg = merge_df["math_score"].mean()

#Average reading score
reading_avg = merge_df["reading_score"].mean()

In [431]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
math_pass_count = merge_df[(merge_df["math_score"] >= 70)].count()["student_name"]
math_pass_rate = math_pass_count / float(total_students) * 100

#% passing reading (the percentage of students who passed reading)
reading_pass_count = merge_df[(merge_df["reading_score"] >= 70)].count()["student_name"]
reading_pass_rate = reading_pass_count / float(total_students) * 100


#% overall passing (the percentage of students who passed math AND reading)
passing_both_count = merge_df[
    (merge_df["math_score"] >= 70) & (merge_df["reading_score"] >= 70)
].count()["student_name"]
overall_pass_rate = passing_both_count /  float(total_students) * 100
overall_pass_rate

65.17232575950983

In [432]:
district_summary = pd.DataFrame({
    "Total_Schools": [total_schools], 
    "Total_Students": [total_students],
    "Total_Budget": [total_budget],
    "Total_Budget": [total_budget],
    "Average_Math_Score": [math_avg], 
    "Average_Reading_Score": [reading_avg], 
    "Passing_Math_Rate": [math_pass_rate],
    "Passing_Reading_Rate": [reading_pass_rate],
    "Overall_Passing_Rate": [overall_pass_rate],
})
# pd.DataFrame(district_summary)

# Formatting
district_summary["Total_Students"] = district_summary["Total_Students"].map("{:,}".format)
district_summary["Total_Budget"] = district_summary["Total_Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Passing_Math_Rate,Passing_Reading_Rate,Overall_Passing_Rate
0,15,"39,170.0","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


<H1>School Summary</H1>

In [433]:
# Use the code provided to select the school type
school_types = schools_df.set_index(["school_name"])["type"]

In [434]:
# Calculate the total student count
per_school_counts = merge_df.groupby("school_name").mean()["size"]

In [435]:
# Calculate the total school budget and per capita spending
per_school_budget = merge_df.groupby("school_name").mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [436]:
# Calculate the average test scores
per_school_math = merge_df.groupby("school_name").mean()["math_score"]
per_school_reading = merge_df.groupby("school_name").mean()["reading_score"]

In [437]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = merge_df[(merge_df["math_score"] >= 70)]

In [438]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = merge_df[(merge_df["reading_score"] >= 70)]

In [439]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = merge_df[
    (merge_df["reading_score"] >= 70) & (merge_df["math_score"] >= 70)]

In [440]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [441]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.

per_school_summary = pd.DataFrame({
    "School_Types": school_types,
    "Student_Count": per_school_counts,
    "Total_School_Budget": per_school_budget,
    "Per_Capita_Budget": per_school_capita,
    "Average_Math_Score": per_school_math, 
    "Average_Reading_Score": per_school_reading, 
    "Passing_Math_Rate": per_school_passing_math,
    "Passing_Reading_Rate": per_school_passing_reading,
    "Overall_Passing_Rate": overall_passing_rate,
    })

#creating my copy of the school spending df before formatting to avoid turning floats to objects
school_spending_df = per_school_summary.copy()
    
# Formatting
per_school_summary["Total_School_Budget"] = per_school_summary["Total_School_Budget"].map("${:,.2f}".format)
per_school_summary["Per_Capita_Budget"] = per_school_summary["Per_Capita_Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School_Types,Student_Count,Total_School_Budget,Per_Capita_Budget,Average_Math_Score,Average_Reading_Score,Passing_Math_Rate,Passing_Reading_Rate,Overall_Passing_Rate
school_name,,,,,,,,,
Bailey High School,District,4976.0,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739.0,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427.0,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917.0,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761.0,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


<H1>Highest Performing Schools (by % Overall Passing)</H1>

In [442]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
per_school_summary.sort_values("Overall_Passing_Rate", ascending = False).head()

,School_Types,Student_Count,Total_School_Budget,Per_Capita_Budget,Average_Math_Score,Average_Reading_Score,Passing_Math_Rate,Passing_Reading_Rate,Overall_Passing_Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635.0,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962.0,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


<H1>Bottom Performing Schools (by % Overall Passing)</H1>

In [443]:
per_school_summary.sort_values("Overall_Passing_Rate").head()

,School_Types,Student_Count,Total_School_Budget,Per_Capita_Budget,Average_Math_Score,Average_Reading_Score,Passing_Math_Rate,Passing_Reading_Rate,Overall_Passing_Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999.0,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917.0,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761.0,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


<H1>Math scores by grade</H1>

In [444]:
# Use the code provided to separate the data by grade
ninth_graders = merge_df[(merge_df["grade"] == "9th")]
tenth_graders = merge_df[(merge_df["grade"] == "10th")]
eleventh_graders = merge_df[(merge_df["grade"] == "11th")]
twelfth_graders = merge_df[(merge_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

# Use the code to select only the `math_score`.
ninth_grader_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "Ninth Grade": ninth_grader_math_scores, 
    "Tenth Grade": tenth_grader_math_scores, 
    "Eleventh Grade": eleventh_grader_math_scores,
    "Twelfth Grade": twelfth_grader_math_scores
})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,Ninth Grade,Tenth Grade,Eleventh Grade,Twelfth Grade
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


<H1>Reading scores by grade</H1>

In [445]:
# Use the code provided to separate the data by grade
ninth_graders = merge_df[(merge_df["grade"] == "9th")]
tenth_graders = merge_df[(merge_df["grade"] == "10th")]
eleventh_graders = merge_df[(merge_df["grade"] == "11th")]
twelfth_graders = merge_df[(merge_df["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

# Use the code to select only the `reading_score`.
ninth_grader_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_grader_reading_scores, 
    "10th": tenth_grader_reading_scores, 
    "11th": eleventh_grader_reading_scores,
    "12th": twelfth_grader_reading_scores
})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


<H1>Scores by school spending</H1>

In [446]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [447]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df.dtypes

School_Types              object
Student_Count            float64
Total_School_Budget      float64
Per_Capita_Budget        float64
Average_Math_Score       float64
Average_Reading_Score    float64
Passing_Math_Rate        float64
Passing_Reading_Rate     float64
Overall_Passing_Rate     float64
dtype: object

In [448]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per_Capita_Budget"], spending_bins, labels=labels, include_lowest=True)

In [449]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average_Math_Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average_Reading_Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Passing_Math_Rate"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Passing_Reading_Rate"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Overall_Passing_Rate"]

In [450]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score by Spending Range": spending_math_scores,
    "Average Reading Score by Spending Range": spending_reading_scores,
    "Math Passing % by Spending Range": spending_passing_math,
    "Reading Passing % by Spending Range": spending_passing_reading,
    "Overall Passing % by Spending Range": overall_passing_spending, 
    })

# Display results
spending_summary

,Average Math Score by Spending Range,Average Reading Score by Spending Range,Math Passing % by Spending Range,Reading Passing % by Spending Range,Overall Passing % by Spending Range
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


<H1>Scores by School Size</H1>

In [451]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [452]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(school_spending_df["Student_Count"], size_bins, labels=labels, include_lowest=True)

In [453]:
list(per_school_summary.columns)

['School_Types',
 'Student_Count',
 'Total_School_Budget',
 'Per_Capita_Budget',
 'Average_Math_Score',
 'Average_Reading_Score',
 'Passing_Math_Rate',
 'Passing_Reading_Rate',
 'Overall_Passing_Rate',
 'School Size']

In [454]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average_Math_Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average_Reading_Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["Passing_Math_Rate"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["Passing_Reading_Rate"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["Overall_Passing_Rate"]

In [455]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score by School Size": size_math_scores,
    "Average Reading Score by School Size": size_reading_scores,
    "Math Passing % by School Size": size_passing_math,
    "Reading Passing % by School Size": size_passing_reading,
    "Overall Passing % by School Size": size_overall_passing, 
    })

# Display results
size_summary

,Average Math Score by School Size,Average Reading Score by School Size,Math Passing % by School Size,Reading Passing % by School Size,Overall Passing % by School Size
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


<H1>Scores by School Type</H1>

In [456]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School_Types"]).mean()
type_reading_scores = per_school_summary.groupby(["School_Types"]).mean()
type_passing_math = per_school_summary.groupby(["School_Types"]).mean()
type_passing_reading = per_school_summary.groupby(["School_Types"]).mean()
type_overall_passing = per_school_summary.groupby(["School_Types"]).mean()

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average_Math_Score"]
average_reading_score_by_type = type_reading_scores["Average_Reading_Score"]
average_percent_passing_math_by_type = type_passing_math["Passing_Math_Rate"]
average_percent_passing_reading_by_type = type_passing_reading["Passing_Reading_Rate"]
average_percent_overall_passing_by_type = type_overall_passing["Overall_Passing_Rate"]

In [457]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score by School Type": average_math_score_by_type,
    "Average Reading Score by School Type": average_reading_score_by_type,
    "Math Passing % by School Type": average_percent_passing_math_by_type,
    "Reading Passing % by School Type": average_percent_passing_reading_by_type,
    "Overall Passing % by School Type": average_percent_overall_passing_by_type, 
    })

# Display results
type_summary

,Average Math Score by School Type,Average Reading Score by School Type,Math Passing % by School Type,Reading Passing % by School Type,Overall Passing % by School Type
School_Types,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
